In [21]:
import pandas as pd
import shapefile
import os

script_dir = os.path.abspath('')
shp_file_path = os.path.join(script_dir, "../data/ny_geo_shape_data/taxi_zones.shp")
target_path = os.path.join(script_dir, "../components/dbt/app/seeds/seed.csv")

def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic["LocationID"]]

        x = (shape.bbox[0] + shape.bbox[2]) / 2
        y = (shape.bbox[1] + shape.bbox[3]) / 2

        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])


sf = shapefile.Reader(shp_file_path)
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]
seed = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")
seed = seed[["LocationID", "borough", "zone", "longitude", "latitude"]]
seed.set_index("LocationID")
seed.to_csv(target_path, index=False)
df_loc = pd.read_csv(target_path)
df_loc.head

<bound method NDFrame.head of      Unnamed: 0  LocationID        borough                     zone  \
0             0           1            EWR           Newark Airport   
1             1           2         Queens              Jamaica Bay   
2             2           3          Bronx  Allerton/Pelham Gardens   
3             3           4      Manhattan            Alphabet City   
4             4           5  Staten Island            Arden Heights   
..          ...         ...            ...                      ...   
266         258         259          Bronx       Woodlawn/Wakefield   
267         259         260         Queens                 Woodside   
268         260         261      Manhattan       World Trade Center   
269         261         262      Manhattan           Yorkville East   
270         262         263      Manhattan           Yorkville West   

     longitude   latitude  
0   -74.171533  40.689483  
1   -73.822478  40.610824  
2   -73.844953  40.865747  
3   -